#### Auth - Drive authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#### Import

In [ ]:
import requests, pandas as pd, numpy as np
from requests import session
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import random
import seaborn as sns
from matplotlib.pyplot import figure
from collections import Counter
from scipy import stats
import decimal
from decimal import *
import powerlaw
from google.colab import files

#### Read data

In [ ]:
network_all = gc.open_by_key('1c5_rTfNYg9NYUiqP28XQo1xALqwZNNUiTdSKqy_zX5Y')
edges_all = network_all.worksheet('edges')
data = edges_all.get_all_values()
df_all = pd.DataFrame(data)
df_all = df_all.rename(columns=df_all.iloc[0]).drop(df_all.index[0])
df_all["Weight"]=df_all["Weight"].str.replace(',','.')
df_all[["Weight"]] = df_all[["Weight"]].apply(pd.to_numeric)

In [ ]:
# 2012
df2012 = df_all.loc[df_all['Timeset'] == '2012']
df2012 = df2012[df2012['Weight'] < 1]

In [ ]:
# 2007
df2007 = df_all.loc[df_all['Timeset'] == '2007']
df2007 = df2007[df2007['Weight'] < 1]

## KS two-sample test

In [ ]:
i=0.00001
a=[]
while i<0.51:
  a.append(i)
  i=i+0.005

#### 2012


In-degree

In [ ]:
stat_all = []
for w in a:
    df_cut = df2012[df2012['Weight'] > w]

    G_cut = nx.DiGraph()
    G_cut.add_nodes_from(df_cut['Source'])
    G_cut.add_nodes_from(df_cut['Target'])
    G_cut.add_edges_from(zip(df_cut['Source'],df_cut['Target']))

    #calculating variables in scale free model (alpha, beta, gamma)
    #top-outdegree
    df_out=pd.DataFrame(sorted(G_cut.out_degree, key=lambda x: x[1], reverse=True))
    df_out.columns = ['ID', 'Out-degree']
    topout=df_out['Out-degree'].iloc[0]
    #top-indegree
    df_in=pd.DataFrame(sorted(G_cut.in_degree, key=lambda x: x[1], reverse=True))
    df_in.columns = ['ID', 'In-degree']
    topin=df_in['In-degree'].iloc[0]
    #topall
    topall=topin+topout
    divid=(topin/topall)+1+(topout/topall)
    alpha=(topin/topall)/divid
    beta=1/divid
    gamma=(topout/topall)/divid

    G_scalefree = nx.scale_free_graph(G_cut.number_of_nodes(), alpha=alpha, beta=beta, gamma=gamma, delta_in=0.6, delta_out=0, seed=1)
    G_random = nx.gnm_random_graph(G_cut.number_of_nodes(), G_cut.number_of_edges(), directed=True, seed=1)

    #in-degree
    degrees_G_cut = [G_cut.in_degree(n) for n in G_cut.nodes()]
    degrees_G_random = [G_random.in_degree(n) for n in G_random.nodes()]
    degrees_G_scalefree = [G_scalefree.in_degree(n) for n in G_scalefree.nodes()]

    ks_random = stats.ks_2samp(degrees_G_cut, degrees_G_random)
    ks_scalefree = stats.ks_2samp(degrees_G_cut, degrees_G_scalefree)

    stat_all.append({'w': w,
                    'nodes':G_cut.number_of_nodes(),
                    'edges':G_cut.number_of_edges(),
                    'ks_random_statistic':ks_random[0],
                    'ks_random_pvalue':decimal.Decimal("{}".format(float((ks_random[1])))),
                    'ks_scalefree_statistic':ks_scalefree[0],
                    'ks_scalefree_pvalue':decimal.Decimal("{}".format(float((ks_scalefree[1])))),
                     })

In [ ]:
df_stat_all = pd.DataFrame(stat_all)
DET_all = gc.open_by_key('1duvehwpb506mSnr6SnJ5iVzvXTwlAzKDp8l-IDnQLUk')
worksheet_DET_all = DET_all.get_worksheet(0)
set_with_dataframe(worksheet_DET_all , df_stat_all)

Out-degree

In [ ]:
stat_all = []
for w in a:
    df_cut = df2012[df2012['Weight'] > w]

    G_cut = nx.DiGraph()
    G_cut.add_nodes_from(df_cut['Source'])
    G_cut.add_nodes_from(df_cut['Target'])
    G_cut.add_edges_from(zip(df_cut['Source'],df_cut['Target']))

    #calculating variables in scale free model (alpha, beta, gamma)
    #top-outdegree
    df_out=pd.DataFrame(sorted(G_cut.out_degree, key=lambda x: x[1], reverse=True))
    df_out.columns = ['ID', 'Out-degree']
    topout=df_out['Out-degree'].iloc[0]
    #top-indegree
    df_in=pd.DataFrame(sorted(G_cut.in_degree, key=lambda x: x[1], reverse=True))
    df_in.columns = ['ID', 'In-degree']
    topin=df_in['In-degree'].iloc[0]
    #topall
    topall=topin+topout
    divid=(topin/topall)+1+(topout/topall)
    alpha=(topin/topall)/divid
    beta=1/divid
    gamma=(topout/topall)/divid

    G_scalefree = nx.scale_free_graph(G_cut.number_of_nodes(), alpha=alpha, beta=beta, gamma=gamma, delta_in=0, delta_out=0.6, seed=1)
    G_random = nx.gnm_random_graph(G_cut.number_of_nodes(), G_cut.number_of_edges(), directed=True, seed=1)

    #out-degree
    degrees_G_cut = [G_cut.out_degree(n) for n in G_cut.nodes()]
    degrees_G_random = [G_random.out_degree(n) for n in G_random.nodes()]
    degrees_G_scalefree = [G_scalefree.out_degree(n) for n in G_scalefree.nodes()]

    ks_random = stats.ks_2samp(degrees_G_cut, degrees_G_random)
    ks_scalefree = stats.ks_2samp(degrees_G_cut, degrees_G_scalefree)

    stat_all.append({'w': w,
                    'nodes':G_cut.number_of_nodes(),
                    'edges':G_cut.number_of_edges(),
                    'ks_random_statistic':ks_random[0],
                    'ks_random_pvalue':decimal.Decimal("{}".format(float((ks_random[1])))),
                    'ks_scalefree_statistic':ks_scalefree[0],
                    'ks_scalefree_pvalue':decimal.Decimal("{}".format(float((ks_scalefree[1])))),
                     })

In [ ]:
df_stat_all = pd.DataFrame(stat_all)
DET_all = gc.open_by_key('1tXishoaXa4x3Q0NtBu2VG7To5fHFy6QYah3oqTXrVRI')
worksheet_DET_all = DET_all.get_worksheet(0)
set_with_dataframe(worksheet_DET_all , df_stat_all)

#### 2007


In [ ]:
i=0.00001
a=[]
while i<0.51:
  a.append(i)
  i=i+0.005

In-degree

In [ ]:
stat_all = []
for w in a:
    df_cut = df2007[df2007['Weight'] > w]

    G_cut = nx.DiGraph()
    G_cut.add_nodes_from(df_cut['Source'])
    G_cut.add_nodes_from(df_cut['Target'])
    G_cut.add_edges_from(zip(df_cut['Source'],df_cut['Target']))

    #calculating variables in scale free model (alpha, beta, gamma)
    #top-outdegree
    df_out=pd.DataFrame(sorted(G_cut.out_degree, key=lambda x: x[1], reverse=True))
    df_out.columns = ['ID', 'Out-degree']
    topout=df_out['Out-degree'].iloc[0]
    #top-indegree
    df_in=pd.DataFrame(sorted(G_cut.in_degree, key=lambda x: x[1], reverse=True))
    df_in.columns = ['ID', 'In-degree']
    topin=df_in['In-degree'].iloc[0]
    #topall
    topall=topin+topout
    divid=(topin/topall)+1+(topout/topall)
    alpha=(topin/topall)/divid
    beta=1/divid
    gamma=(topout/topall)/divid

    G_scalefree = nx.scale_free_graph(G_cut.number_of_nodes(), alpha=alpha, beta=beta, gamma=gamma, delta_in=0.6, delta_out=0, seed=1)
    G_random = nx.gnm_random_graph(G_cut.number_of_nodes(), G_cut.number_of_edges(), directed=True, seed=1)

    #in-degree
    degrees_G_cut = [G_cut.in_degree(n) for n in G_cut.nodes()]
    degrees_G_random = [G_random.in_degree(n) for n in G_random.nodes()]
    degrees_G_scalefree = [G_scalefree.in_degree(n) for n in G_scalefree.nodes()]

    ks_random = stats.ks_2samp(degrees_G_cut, degrees_G_random)
    ks_scalefree = stats.ks_2samp(degrees_G_cut, degrees_G_scalefree)

    stat_all.append({'w': w,
                    'nodes':G_cut.number_of_nodes(),
                    'edges':G_cut.number_of_edges(),
                    'ks_random_statistic':ks_random[0],
                    'ks_random_pvalue':decimal.Decimal("{}".format(float((ks_random[1])))),
                    'ks_scalefree_statistic':ks_scalefree[0],
                    'ks_scalefree_pvalue':decimal.Decimal("{}".format(float((ks_scalefree[1])))),
                     })

In [ ]:
df_stat_all = pd.DataFrame(stat_all)
DET_all = gc.open_by_key('1z3yya39BXZuyKikSsF4FzVZnuqKPwuXey8TQTpnGbh0')
worksheet_DET_all = DET_all.get_worksheet(0)
set_with_dataframe(worksheet_DET_all , df_stat_all)

Out-degree

In [ ]:
stat_all = []
for w in a:
    df_cut = df2007[df2007['Weight'] > w]

    G_cut = nx.DiGraph()
    G_cut.add_nodes_from(df_cut['Source'])
    G_cut.add_nodes_from(df_cut['Target'])
    G_cut.add_edges_from(zip(df_cut['Source'],df_cut['Target']))

    #calculating variables in scale free model (alpha, beta, gamma)
    #top-outdegree
    df_out=pd.DataFrame(sorted(G_cut.out_degree, key=lambda x: x[1], reverse=True))
    df_out.columns = ['ID', 'Out-degree']
    topout=df_out['Out-degree'].iloc[0]
    #top-indegree
    df_in=pd.DataFrame(sorted(G_cut.in_degree, key=lambda x: x[1], reverse=True))
    df_in.columns = ['ID', 'In-degree']
    topin=df_in['In-degree'].iloc[0]
    #topall
    topall=topin+topout
    divid=(topin/topall)+1+(topout/topall)
    alpha=(topin/topall)/divid
    beta=1/divid
    gamma=(topout/topall)/divid

    G_scalefree = nx.scale_free_graph(G_cut.number_of_nodes(), alpha=alpha, beta=beta, gamma=gamma, delta_in=0, delta_out=0.6, seed=1)
    G_random = nx.gnm_random_graph(G_cut.number_of_nodes(), G_cut.number_of_edges(), directed=True, seed=1)

    #out-degree
    degrees_G_cut = [G_cut.out_degree(n) for n in G_cut.nodes()]
    degrees_G_random = [G_random.out_degree(n) for n in G_random.nodes()]
    degrees_G_scalefree = [G_scalefree.out_degree(n) for n in G_scalefree.nodes()]

    ks_random = stats.ks_2samp(degrees_G_cut, degrees_G_random)
    ks_scalefree = stats.ks_2samp(degrees_G_cut, degrees_G_scalefree)

    stat_all.append({'w': w,
                    'nodes':G_cut.number_of_nodes(),
                    'edges':G_cut.number_of_edges(),
                    'ks_random_statistic':ks_random[0],
                    'ks_random_pvalue':decimal.Decimal("{}".format(float((ks_random[1])))),
                    'ks_scalefree_statistic':ks_scalefree[0],
                    'ks_scalefree_pvalue':decimal.Decimal("{}".format(float((ks_scalefree[1])))),
                     })

In [ ]:
df_stat_all = pd.DataFrame(stat_all)
DET_all = gc.open_by_key('15q7qktebfX55LigsikT79yqLG2jb3hcWnyigNeKT8_A')
worksheet_DET_all = DET_all.get_worksheet(0)
set_with_dataframe(worksheet_DET_all , df_stat_all)